## Introduction to Automation at Scale

Automating with Configuration Management

Scale : Being able to scale what we do means that we can keep achieving larger impacts with the same amount of effort when a system scales.
a scalable system is a flexible one
 
These concepts are the building blocks for letting us manage a growing number of devices without having to grow the team in charge of them.

Puppet : The configuration management tool

Knowing how to automatically manage the configuration of the devices in your fleet will let your team handle a lot more work with the same amount of people. It also frees up time to do more interesting stuff since all the boring tasks can get automated.

Automation is an essential tool for keeping up with the infrastructure needs of a growing business.

Configuration Management : how we can use configuration management to maintain the computers in our fleet
It means using a configuration management system to handle all of the configuration of the devices in your fleet, also known as nodes. 
A configuration management tool can take the rules you define and apply them to the systems that it manages, making changes efficient and consistent. 

Unmanaged Configuration : By manually deploying the installation and configuring the computer

Configuration Management Tool : Puppet, Chef, Ansible, CFEngine
Puppet : it's the current industry standard for configuration management

Infrastructure as code (IaC) : how we can all benefit from treating our infrastructure as code
This means that we can model the behavior of our IT infrastructure in files that can be processed by automatic tools. These files can then be tracked in a version control system.
The paradigm of storing all the configuration for the managed devices in version controlled files
when all of the configuration necessary to deploy and manage a node in the infrastructure is stored in version control

The principals of Infrastructure as Code are commonly applied in cloud computing environments, where machines are treated like interchangeable resources, instead of individual computers. This principle is also known as treating your computers as cattle instead of pets because you care for them as a group rather than individually.

One valuable benefit of this process is that the configuration applied to the device doesn't depend on a human remembering to follow all the necessary steps.
Rest assured, silly human, the result will always be the same, making the deployment consistent.

To sum all of this up, managing your Infrastructure as Code it means that your fleet of nodes are 
1) consistent, 
2) versioned, 
3) reliable, 
4) and repeatable.

Viewing your infrastructure in this way helps your IT team adapt and stay flexible. The technology industry is constantly changing and evolving. Automation and configuration management can help you embrace that change instead of avoiding it

## Introduction to Puppet

Puppet is the current industry standard for managing the configuration of computers in a fleet of machines.
Cross platform, Open source

We typically deploy puppet using a client-server architecture. The client is known as the Puppet agent, and the service is known as the Puppet master. When using this model, the agent connects to the master and sends a bunch of facts that describe the computer to the master. The master then processes this information, generates the list of rules that need to be applied on the device, and sends this list back to the agent. The agent is then in charge of making any necessary changes on the computer.

#Example rule

class sudo{

    package { 'sudo':
        ensure => present,
    }
}

Package management system
Linux : APT, YUM, DNF
MacOS : Apple Provider, MacPorts
Windows : we'll need to add an extra attribute to our rule, stating where the installer file is located on the local desk or a network mounted resource. Puppet will then execute the installer and make sure that it finishes successfully. If you use Chocolatey to manage your windows packages, you can add an extra Chocolatey provider to Puppet to support that

Puppet Resources
Resources : are the basic unit for modeling the configuration that we want to manage.

class sysctl {
    # Make sure the directory exists, some distros don't have it
    file { '/etc/sysctl.d':
        ensure => directory,
    }
    
}

class timezone{
    
    file { '/etc/timezone':
        ensure => file,
        content => "UTC\n",
        replace => True,
    }
    
}

How do these rules turn into changes in our computers? When we declare a resource in our puppet rules. We're defining the desired state of that resource in the system. The puppet agent then turns the desired state into reality using providers.

Puppet Classes

class ntp {
    package { 'ntp':
        ensure => latest,
    }
    file { '/etc/ntp.conf':
        source => 'puppet:///modules/ntp/ntp.conf'
        replace => true,
    }
    service { 'ntp':
        enable => true,
        ensure => running,
    }
}

In this case, we have a class with three resources, a package, a file, and a service. All of them are related to the Network Time Protocol, or NTP, the mechanism our computers use to synchronize the clocks. 

## The Building Blocks of Configuration Management

Domain-Specific Languages (DSL) : A programming language that's more limited in scope

In the case of Puppet, the DSL is limited to operations related to when and how to apply configuration management rules to our devices.

On top of the basic resource types that we already checked out, Puppet's DSL includes 
1) variables
2) conditional statements
3) functions

Puppet facts : Facts are variables that represent the characteristics of the system. 
When the Puppet agent runs, it calls a program called factor which analyzes the current system, storing the information it gathers in these facts. Once it's done, it sends the values for these facts to the server, which uses them to calculate the rules that should be applied. Puppet comes with a bunch of baked-in core facts that store useful information about the system like what the current OS is, how much memory the computer has whether it's a virtual machine or not or what the current IP address is. If the information we need to make a decision isn't available through one of these facts, we can also write a script that checks for the information and turns it into our own custom fact.

if $facts['is_virtual'] {
    package { 'smartmontools' :
        ensure => purged,
    
    }
}   else {
    package { 'smartmontools' :
        ensure => installed,
    }
}

In particular, the facts variable is what's known as a hash in the Puppet DSL, which is equivalent to a dictionary in Python.

class class_name {
    resource_type { 'title' :
        attribute -> value,
    }
    
}

The Driving Principles of Configuration Management

Puppet uses a declarative language because we declare the state that we want to achieve rather than the steps to get there
Traditional languages like Python or C are called procedural because we write out the procedure that the computer needs to follow to reach our desired goal. 

Just remember that when it comes to configuration management, it makes sense to simply state what the configuration should be, not what the computer should do to get there

Idempotent action can be performed over and over again without changing the system after the first time the action was performed, and with no unintended side effects.
If a script is idempotent, it means that it can fail halfway through its task and be run again without problematic consequences.

Say you're running your configuration management system to setup a new server. Unfortunately, the setup fails because you forgot to add a second disk to the computer and the configuration required two disks. If your automation is idempotent, you can add the missing disk and then have the system pick up from where it left off. 



file { '/etc/issue/':
    mode => '0644',
    content => "Internal system \l \n",
}

ls -l example.txt
mv example.txt Desktop/
mv example.txt Desktop/ -> No such file or directory

An exception to this is the exec resource, which runs commands for us. The actions taken by the exec resource might not be idempotent since a command might modify the system each time it's executed.

So if we need to use the exec resource to run a command for us, we need to be careful to ensure that the action is idempotent. We could do that for example by using the onlyif attribute

exec { 'move example file':
    command => 'mv /home/user/example.txt /home/user/Desktop',
    onlyif => 'test -e /home/user/example.txt',
}

Test and repair paradigm : This means that actions are taken only when they are necessary to achieve a goal. Puppet will first test to see if the resource being managed like a file or a package, actually needs to be modified. If the file exists in the place we want it to, no action needs to be taken.

Puppet is stateless : this means that there's no state being kept between runs of the agent.Each Puppet run is independent of the previous one, and the next one.